# Constraint Optimization


**约束优化，也可称为“约束规划（Constraint Programming）”**，是指从非常多的候选方案中识别和选择可行解决方案的方法，其中问题可以根据任意约束进行建模。CP问题出现在许多应用工程学科中。

- **它基于可行性，找到可行的解决方案，而不完全是优化**（找到最优解决方案），并且更侧重于约束和变量而不是目标函数。事实上，CP问题甚至可能没有目标函数。使用CP是为了通过向问题添加约束，将一大堆可能的解决方案缩小到更易于管理的子集。

- 常见的约束优化问题包括员工排程、作业车间调度等。还有一些平时比较熟悉的算法问题，比如N皇后、数独求解等也可归纳到这一范畴中。


--------

关于求解器本身不再赘述，Ortools提供了几种解决CP问题的方法：
1. CP-SAT 求解器：使用SAT（满足性）方法的约束规划求解器，这是在原始CP求解器基础上迭代的新版本；
2. 原始的CP求解器。

P.S. 如果可以使用线性目标和线性约束对问题进行建模，那么可以考虑 **MPSolver**。ortools官网同样补充，遇到路径问题等，通常使用专门的车辆路径库来解决（即使它们可以用线性模型表示）。



In [6]:
"""OR-Tools solution to the N-queens problem."""
import time
from ortools.sat.python import cp_model
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print(
            f"Solution {self.__solution_count}, "
            f"time = {current_time - self.__start_time} s"
        )
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i.
                    print("Q", end=" ")
                else:
                    print("_", end=" ")
            print()
        print()


def main(board_size):
    model = cp_model.CpModel() # 创建求解器

    # 创建变量，变量的下标就是其所处的列，变量的值就是其所在的行
    queens = [model.NewIntVar(0, \
        board_size - 1, f"x_{i}") for i in range(board_size)]

    # 约束条件
    model.AddAllDifferent(queens)

    # 对角线不能有冲突
    model.AddAllDifferent(queens[i] + i for i in range(board_size))
    model.AddAllDifferent(queens[i] - i for i in range(board_size))

    # 模型求解
    solver = cp_model.CpSolver()
    solution_printer = NQueenSolutionPrinter(queens)
    solver.parameters.enumerate_all_solutions = True
    solver.Solve(model, solution_printer)

    print("\nStatistics")
    print(f"  conflicts      : {solver.NumConflicts()}")
    print(f"  branches       : {solver.NumBranches()}")
    print(f"  wall time      : {solver.WallTime()} s")
    print(f"  solutions found: {solution_printer.solution_count()}")

if __name__ == "__main__":
    # 这里求解一个 11 皇后问题
    size = 11
    main(size)

Solution 0, time = 0.00818777084350586 s
_ _ _ _ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ _ _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 

Solution 1, time = 0.008852005004882812 s
_ _ _ _ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 

Solution 2, time = 0.010380029678344727 s
_ _ _ _ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ _ _ _ 

Solution 3, time = 0.011183023452758789 s
_ _ _ _ _ _ Q _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ 

## Model a Sudoku

> src: https://www.kaggle.com/code/pintowar/modeling-a-sudoku-solver-with-or-tools
> 
> src: http://www.hakank.org/google_or_tools/


1. A Standard Sudoku 
2. An Alphadoku, 25 by 25
3. A Killer Sudoku

In [14]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def main(grid, grid_size):

    model = cp.CpModel()
    n = grid_size
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i * grid_size  + j] != "0":
                x[i, j] = int(grid[i * grid_size + j]) 
                # 这里规定：初始已经给了的数字不变
            else:
                x[i, j] = model.NewIntVar(1, grid_size, 'x[{},{}]'.format(i,j) ) 
                # 这里规定：余下的数字从1到9
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)

    # 3 * 3 的宫内数字不能重复
    for i in range(2):
        for j in range(2):
            cell = [
                x[r, c]
                for r in range(i * 3, i * 3 + 3)
                for c in range(j * 3, j * 3 + 3)
            ]
            model.AddAllDifferent(cell)
    solver = cp.CpSolver() 
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    main("549001738367008001200073040000900005000705460135840070004000307780350006023080000", 9)

5 4 9 2 6 1 7 3 8 
3 6 7 4 9 8 5 2 1 
2 1 8 5 7 3 6 4 9 
4 7 6 9 3 2 8 1 5 
8 9 2 7 1 5 4 6 3 
1 3 5 8 4 6 9 7 2 
6 5 4 1 2 9 3 8 7 
7 8 1 3 5 4 2 9 6 
9 2 3 6 8 7 1 5 4 

NumConflicts: 0
NumBranches: 0
WallTime: 0.004234


In [28]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

str2Int = {
    "A" : 1,
    "B" : 2,
    "C" : 3,
    "D" : 4,
    "E" : 5,
    "F" : 6,
    "G" : 7,
    "H" : 8,
    "I" : 9,
    "J": 10,
    "K": 11,
    "L": 12,
    "M": 13,
    "N": 14,
    "O": 15,
    "P": 16,
    "Q": 17,
    "R": 18,
    "S": 19,
    "T": 20,
    "U": 21,
    "V": 22,
    "W": 23,
    "X": 24,
    "Y": 25,
}

def main_25by25(grid, grid_size):

    model = cp.CpModel()
    n = grid_size
    x = {}
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i * grid_size  + j] != "0":
                x[i, j] = str2Int[grid[i * grid_size + j]]
                # 这里规定：初始已经给了的数字不变
            else:
                x[i, j] = model.NewIntVar(1, grid_size, 'x[{},{}]'.format(i,j) ) 
                # 这里规定：余下的数字从1到9
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)

    # 25 * 25 的宫内数字不能重复
    for i in range(5):
        for j in range(5):
            cell = [
                x[r, c]
                for r in range(i * 5, i * 5 + 5)
                for c in range(j * 5, j * 5 + 5)
            ]
            model.AddAllDifferent(cell)

    solver = cp.CpSolver() 
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    grid = "0G0B0UJ00H00000E00KVFR0PM0UEPSG0Q00O00000000W00I000000VXWTA00JCMR000PD0Y00B0WO00000ME0DK00X00J0AGNSQ0AFHL0B0DOEY00PI0CGN00000T000CP0OQ00000V0WRM0E0HI0E00UQ0GXWNY000000FB00T0K0000N000D00LQJ0UK0T0I0OR00VI00R00AEBMTOD0N000Q0S0J0K0XO0I0R0FS0000GV00UBN0MPS00TPNQ000J0LHOWX0Y0000D0DCG0000W0000SN000K0FQVE0000JLBH0000R0P0T0000OKWY0000MXYA0K000FB0000N0000SHC0N0000E0RLVWA0D000ICPX00JHT0MGV00PW0000AU0Y0L0JB0E0D0W0E000Y0UMKCQGI00N00TF00UJ0F0H0DT0EYB00P000Q0000F0Q00MG000000WBEJS0LA00D0PV0O0NSB0D00000CX0TM000H00000BSY0RX00LHCM0T0VEJQ0IYQRE0A00V00GT0PB00000DN0A00S0OF000CVRP00NDQEW000000H00D00000000M00A0KTFPR0CL0DNJP00G00000H00XR0B0A0"
    main_25by25(grid, 25)

24 7 25 2 4 21 10 14 3 8 1 12 9 23 19 5 20 17 11 22 6 18 15 16 13 
10 21 5 16 19 7 25 17 6 11 15 8 20 24 14 13 1 2 18 23 3 4 9 22 12 
17 11 14 9 22 24 23 20 1 19 7 10 3 13 18 6 12 15 16 4 8 25 21 5 2 
18 23 15 3 20 12 9 16 13 5 2 4 11 22 6 24 8 21 10 25 1 7 14 19 17 
13 1 6 8 12 18 2 22 4 15 5 25 17 21 16 9 19 3 7 14 10 11 23 24 20 
20 19 4 7 3 16 12 15 17 10 11 2 14 6 22 1 23 18 13 24 5 21 8 9 25 
5 13 1 21 17 3 7 24 23 14 25 16 8 18 9 10 15 6 2 19 4 20 12 11 22 
16 2 23 14 6 19 8 4 22 13 12 17 10 7 21 11 25 20 5 9 24 15 18 3 1 
22 9 12 25 18 11 21 1 5 2 13 20 15 4 24 14 16 8 3 17 7 19 6 10 23 
11 8 24 15 10 9 20 18 25 6 19 3 23 1 5 7 22 12 4 21 2 14 17 13 16 
19 22 18 20 16 14 17 6 7 3 10 11 12 8 15 23 24 5 25 2 9 13 1 4 21 
4 3 7 1 9 13 15 23 20 16 21 24 19 14 25 12 10 11 8 6 17 22 5 2 18 
21 5 10 12 2 8 22 9 19 24 18 13 16 3 20 4 17 7 1 15 11 23 25 6 14 
23 17 13 24 25 1 4 11 10 21 9 6 2 5 7 20 18 14 22 16 15 12 19 8 3 
15 14 11 6 8 25 5 2 18 12 22 23 1 17 4 19 21 13 9 3 16 24 20 7

![Alphabetic Sudoku](./assets/figures/Board.png)

## Solve Killer Sudoku 

In [9]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

def calc(model, cc, x, res):
    # 该函数让Model中增加一个约束：每个小宫内数字和为res，小宫内数字不重复
    cage = [x[i[0] - 1, i[1] - 1] for i in cc]
    model.Add(sum(cage) == res)
    model.AddAllDifferent(cage)

def main():

    model = cp.CpModel()
    n = 9
    # 规定杀手数独的特殊表示方法： http://en.wikipedia.org/wiki/File:Killersudoku_color.svg
    # list( 小的宫内数字和, [ 宫内元素的坐标 ] )

    problem = [[3, [[1, 1], [1, 2]]], [15, [[1, 3], [1, 4], [1, 5]]],
                [22, [[1, 6], [2, 5], [2, 6], [3, 5]]], [4, [[1, 7], [2, 7]]],
                [16, [[1, 8], [2, 8]]], [15, [[1, 9], [2, 9], [3, 9], [4, 9]]],
                [25, [[2, 1], [2, 2], [3, 1], [3, 2]]], [17, [[2, 3], [2, 4]]],
                [9, [[3, 3], [3, 4], [4, 4]]], [8, [[3, 6], [4, 6], [5, 6]]],
                [20, [[3, 7], [3, 8], [4, 7]]], [6, [[4, 1], [5, 1]]],
                [14, [[4, 2], [4, 3]]], [17, [[4, 5], [5, 5], [6, 5]]],
                [17, [[4, 8], [5, 7], [5, 8]]], [13, [[5, 2], [5, 3], [6, 2]]],
                [20, [[5, 4], [6, 4], [7, 4]]], [12, [[5, 9], [6, 9]]],
                [27, [[6, 1], [7, 1], [8, 1], [9, 1]]],
                [6, [[6, 3], [7, 2], [7, 3]]], [20, [[6, 6], [7, 6], [7, 7]]],
                [6, [[6, 7], [6, 8]]], [10, [[7, 5], [8, 4], [8, 5], [9, 4]]],
                [14, [[7, 8], [7, 9], [8, 8], [8, 9]]], [8, [[8, 2], [9, 2]]],
                [16, [[8, 3], [9, 3]]], [15, [[8, 6], [8, 7]]],
                [13, [[9, 5], [9, 6], [9, 7]]], [17, [[9, 8], [9, 9]]]]

    x = {}
    for i in range(n):
        for j in range(n):
            x[i, j] = model.NewIntVar(1, n, "x[%i,%i]" % (i, j))

    x_flat = [x[i, j] for i in range(n) for j in range(n)]
    
    # ======= 下面是约束 ==========
    # 行和列必须不重复
    for i in range(n):
        row = [x[i, j] for j in range(n)]
        model.AddAllDifferent(row)
        
        col = [x[j, i] for j in range(n)]
        model.AddAllDifferent(col)

    # 3 * 3 的宫内数字不能重复
    for i in range(2):
        for j in range(2):
            cell = [
                x[r, c]
                for r in range(i * 3, i * 3 + 3)
                for c in range(j * 3, j * 3 + 3)
            ]
            model.AddAllDifferent(cell)

    # 小宫内的约束
    for (res, segment) in problem:
        calc(model,segment, x, res)
        
    solver = cp.CpSolver() 
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        for i in range(n):
            for j in range(n):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())


if __name__ == "__main__":
    main()

2 1 5 6 4 7 3 9 8 
3 6 8 9 5 2 1 7 4 
7 9 4 3 8 1 6 5 2 
5 8 6 2 7 4 9 3 1 
1 4 2 5 9 3 8 6 7 
9 7 3 8 1 6 4 2 5 
8 2 1 7 3 9 5 4 6 
6 5 9 4 2 8 7 1 3 
4 3 7 1 6 5 2 8 9 

NumConflicts: 0
NumBranches: 0
WallTime: 0.011290000000000001


![Killer Sudoku](./assets/figures/KillerSudoku.png)

## 解决加密难题

In [1]:
"""Cryptarithmetic puzzle.

First attempt to solve equation CP + IS + FUN = TRUE
where each letter represents a unique digit.

This problem has 72 different solutions in base 10.
"""
from ortools.constraint_solver import pywrapcp


def main():
    # Constraint programming engine
    solver = pywrapcp.Solver("CP is fun!")

    base = 10

    # Decision variables.
    digits = list(range(0, base))
    digits_without_zero = list(range(1, base))
    c = solver.IntVar(digits_without_zero, "C")
    p = solver.IntVar(digits, "P")
    i = solver.IntVar(digits_without_zero, "I")
    s = solver.IntVar(digits, "S")
    f = solver.IntVar(digits_without_zero, "F")
    u = solver.IntVar(digits, "U")
    n = solver.IntVar(digits, "N")
    t = solver.IntVar(digits_without_zero, "T")
    r = solver.IntVar(digits, "R")
    e = solver.IntVar(digits, "E")

    # We need to group variables in a list to use the constraint AllDifferent.
    letters = [c, p, i, s, f, u, n, t, r, e]

    # Verify that we have enough digits.
    assert base >= len(letters)

    # Define constraints.
    solver.Add(solver.AllDifferent(letters))

    # CP + IS + FUN = TRUE
    solver.Add(
        p + s + n + base * (c + i + u) + base * base * f
        == e + base * u + base * base * r + base * base * base * t
    )

    solution_count = 0
    db = solver.Phase(letters, solver.INT_VAR_DEFAULT, solver.INT_VALUE_DEFAULT)
    solver.NewSearch(db)
    while solver.NextSolution():
        print(letters)
        # Is CP + IS + FUN = TRUE?
        assert (
            base * c.Value()
            + p.Value()
            + base * i.Value()
            + s.Value()
            + base * base * f.Value()
            + base * u.Value()
            + n.Value()
            == base * base * base * t.Value()
            + base * base * r.Value()
            + base * u.Value()
            + e.Value()
        )
        solution_count += 1
    solver.EndSearch()
    print(f"Number of solutions found: {solution_count}")


if __name__ == "__main__":
    main()

[C(2), P(3), I(7), S(4), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(3), I(7), S(5), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(5), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(6), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(3), I(7), S(8), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(3), I(7), S(8), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(3), F(9), U(6), N(8), T(1), R(0), E(5)]
[C(2), P(4), I(7), S(8), F(9), U(6), N(3), T(1), R(0), E(5)]
[C(2), P(5), I(7), S(3), F(9), U(4), N(8), T(1), R(0), E(6)]
[C(2), P(5), I(7), S(3), F(9), U(8), N(6), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(6), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(5), I(7), S(8), F(9), U(4), N(3), T(1), R(0), E(6)]
[C(2), P(6), I(7), S(3), F(9), U(8), N(5), T(1), R(0), E(4)]
[C(2), P(6), I(7), S(5), F(9), U(8), N(3), T(1), R(0), E(4)]
[C(2), P(8), I(7), S(3), F(9), U(4), N(5), T(1), R(0), E(6)]
[C(2), P(8), I(7), S(3), F(9), U(6), N(4), T(1), R(0), E(5)]
[C(2), P(8), I(7), S(4),